In [40]:
import minsearch

In [41]:
import json 

with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [42]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [43]:
documents[10]

{'text': 'It depends on your background and previous experience with modules. It is expected to require about 5 - 15 hours per week. [source1] [source2]\nYou can also calculate it yourself using this data and then update this answer.',
 'section': 'General course-related questions',
 'question': 'Course - \u200b\u200bHow many hours per week am I expected to spend on this  course?',
 'course': 'data-engineering-zoomcamp'}

In [44]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [45]:
q = 'the course has already started, can I still enroll?'

In [46]:
index.fit(documents)

In [47]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query = q, 
    filter_dict = {'course': 'data-engineering-zoomcamp'}, 
    boost_dict = boost, 
    num_results = 5
)

In [48]:
from openai import OpenAI

In [49]:
import os
os.environ['OPENAI_API_KEY'] = 'my-openai-key'

In [50]:
client = OpenAI()

In [51]:
response = client.chat.completions.create(
    model = 'gpt-4o', 
    messages=[{"role":"user", "content":q}]
)

In [52]:
response.choices[0].message.content

"It's best to check directly with the institution or organization offering the course. Policies vary widely: some courses allow late enrollment within a certain period after the start date, while others may not. Reach out to the course administrator or look for information on their website. If they do allow late enrollments, make sure to catch up on any missed materials to stay on track with the class."

In [62]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION. 
If the context doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT: {context}
""".strip()

In [59]:
context = ""

for doc in results: 
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer:{doc['text']}\n\n"

In [61]:
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer:Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer:Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - When will the course start?
answer:The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with t

In [66]:
prompt = prompt_template.format(question = q, context = context).strip()

In [67]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION. 
If the context doesn't contain the answer, output NONE

QUESTION: the course has already started, can I still enroll?

CONTEXT: section: General course-related questions
question: Course - Can I still join the course after the start date?
answer:Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer:Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your fin

In [68]:
response = client.chat.completions.create(
    model = 'gpt-4o', 
    messages=[{"role":"user", "content":prompt}]
)

In [69]:
response.choices[0].message.content

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."